# Building AI Agents with Strands and Mistral on Amazon Bedrock

---

[Strands Agents](https://github.com/strands-agents/sdk-python) is an open-source SDK from AWS for building production-ready AI agent systems. Combined with Mistral's powerful models on Amazon Bedrock, you can create sophisticated conversational agents with just a few lines of code.

## What You'll Learn

This notebook demonstrates Strands Agents capabilities with Mistral models:

1. **Basic Agents** - Create agents with different Mistral models
2. **Model Comparison** - Compare response quality and latency
3. **Streaming Responses** - Real-time response handling
4. **Multi-Turn Conversations** - Context-aware dialogue
5. **Hooks & Callbacks** - Event-driven monitoring
6. **Session Persistence** - Save and resume conversations
7. **Multi-Agent Orchestration** - Chain agents for complex workflows
8. **Practical Use Cases** - Expert panel and content pipeline

## Mistral Models Used

| Model | Parameters | Best For |
|-------|------------|----------|
| **Mistral Large 3** | 675B | Orchestration, complex reasoning |
| **Ministral 3B** | 3B | Quick classification, routing |
| **Ministral 8B** | 8B | Balanced tasks |
| **Ministral 14B** | 14B | Detailed analysis, code generation |

---

## 1. Setup

---

In [ ]:
# Install Strands Agents SDK
%pip install --upgrade --quiet strands-agents

In [ ]:
import json
import time

from strands import Agent
from strands.models.bedrock import BedrockModel

# Mistral Model IDs on Amazon Bedrock
MODELS = {
    "large": "mistral.mistral-large-3-675b-instruct",
    "3B": "mistral.ministral-3-3b-instruct",
    "8B": "mistral.ministral-3-8b-instruct",
    "14B": "mistral.ministral-3-14b-instruct"
}

REGION = "us-west-2"

print("Mistral Models Available:")
for name, model_id in MODELS.items():
    print(f"  {name}: {model_id}")

In [ ]:
def get_model(model_key: str, **kwargs) -> BedrockModel:
    """Create a BedrockModel instance for the specified Mistral model."""
    return BedrockModel(
        model_id=MODELS[model_key],
        region_name=REGION,
        **kwargs
    )

# Test connection
test_model = get_model("8B")
print(f"Connected to Bedrock in {REGION}")

---

## 2. Basic Agent Creation

Creating an agent with Strands is simple - just specify a model and optionally a system prompt.

---

In [ ]:
# Create a simple agent with Ministral 8B
agent = Agent(
    model=get_model("8B"),
    system_prompt="You are a helpful assistant. Keep responses concise (2-3 sentences max)."
)

# Simple conversation
response = agent("What is the capital of France and why is it significant?")
print(f"Response: {response}")

In [ ]:
# Compare responses across model sizes
test_prompt = "Explain what an API is in one sentence, suitable for a beginner."

print("Model Comparison")
print("=" * 70)
print(f"Prompt: {test_prompt}")
print("=" * 70)

for model_name in ["3B", "8B", "14B"]:
    agent = Agent(
        model=get_model(model_name),
        system_prompt="Give extremely concise answers in one sentence."
    )
    
    start = time.time()
    response = agent(test_prompt)
    latency = time.time() - start
    
    print(f"\n--- Ministral {model_name} ({latency:.2f}s) ---")
    print(response)

---

## 3. Streaming Responses

Stream responses in real-time using callbacks or async iterators.

---

In [ ]:
from strands.handlers.callback_handler import PrintingCallbackHandler

# Agent with callback handler for streaming output
streaming_agent = Agent(
    model=get_model("8B"),
    callback_handler=PrintingCallbackHandler(),
    system_prompt="You are a creative storyteller."
)

# This will stream the response as it's generated
print("Streaming response:")
print("-" * 50)
response = streaming_agent("Write a very short story (3 sentences) about a robot learning to paint.")

In [ ]:
# Async streaming with stream_async()
async def stream_response():
    agent = Agent(
        model=get_model("8B"),
        system_prompt="You explain concepts clearly and concisely."
    )
    
    print("Async streaming:")
    print("-" * 50)
    
    async for event in agent.stream_async("What is quantum computing in 2 sentences?"):
        # Events are dictionaries with 'data' key for text deltas
        if "data" in event and event.get("data"):
            print(event["data"], end="", flush=True)
    print()

await stream_response()

---

## 4. Multi-Turn Conversations

Strands agents maintain conversation history automatically.

---

In [ ]:
# Create an agent for multi-turn conversation
chat_agent = Agent(
    model=get_model("8B"),
    system_prompt="""You are a helpful AI tutor teaching Python programming.
    Remember what the student has learned in previous messages.
    Keep explanations brief and build on prior knowledge."""
)

# First turn
print("Turn 1:")
print("-" * 40)
response1 = chat_agent("What is a variable in Python?")
print(response1)

In [ ]:
# Second turn - agent remembers context
print("Turn 2:")
print("-" * 40)
response2 = chat_agent("Can you show me an example?")
print(response2)

In [ ]:
# Third turn - building on previous knowledge
print("Turn 3:")
print("-" * 40)
response3 = chat_agent("What's the difference between that and a constant?")
print(response3)

In [ ]:
# View conversation history
print(f"\nConversation has {len(chat_agent.messages)} messages")
for i, msg in enumerate(chat_agent.messages):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])
    if content and isinstance(content, list) and len(content) > 0:
        text = content[0].get('text', '')[:60]
        if len(content[0].get('text', '')) > 60:
            text += '...'
        print(f"  {i+1}. [{role}]: {text}")

---

## 5. Hooks and Callbacks

Implement custom hooks for logging, monitoring, and extending agent behavior.

---

In [ ]:
from strands.hooks import HookProvider, HookRegistry
from strands.hooks import (
    BeforeInvocationEvent,
    AfterInvocationEvent
)


class MetricsHook(HookProvider):
    """Custom hook for tracking agent metrics."""
    
    def __init__(self):
        self.invocation_count = 0
        self.total_latency = 0
        self.start_time = None
    
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeInvocationEvent, self.on_invocation_start)
        registry.add_callback(AfterInvocationEvent, self.on_invocation_end)
    
    def on_invocation_start(self, event: BeforeInvocationEvent) -> None:
        self.invocation_count += 1
        self.start_time = time.time()
        print(f"[METRICS] Invocation #{self.invocation_count} started")
    
    def on_invocation_end(self, event: AfterInvocationEvent) -> None:
        duration = time.time() - self.start_time
        self.total_latency += duration
        print(f"[METRICS] Completed in {duration:.2f}s")
    
    def get_stats(self) -> dict:
        avg_latency = self.total_latency / self.invocation_count if self.invocation_count > 0 else 0
        return {
            "total_invocations": self.invocation_count,
            "total_latency": f"{self.total_latency:.2f}s",
            "avg_latency": f"{avg_latency:.2f}s"
        }


# Create metrics hook
metrics = MetricsHook()

# Agent with hooks
monitored_agent = Agent(
    model=get_model("8B"),
    hooks=[metrics],
    system_prompt="You are a helpful assistant. Be concise (1-2 sentences)."
)

print("Monitored agent created with custom hooks")

In [ ]:
# Test the monitored agent multiple times
questions = [
    "What is 2+2?",
    "Name 3 planets.",
    "What color is the sky?"
]

for q in questions:
    print(f"\nQ: {q}")
    response = monitored_agent(q)
    print(f"A: {response}")

print(f"\n--- Final Metrics ---")
print(json.dumps(metrics.get_stats(), indent=2))

---

## 6. Conversation Persistence

Save and restore agent conversation history for stateful assistants.

---

In [ ]:
import tempfile
import os

# Create a temporary directory for storing conversations
session_dir = tempfile.mkdtemp(prefix="strands_sessions_")
print(f"Session directory: {session_dir}")

# Create agent for conversation
persistent_agent = Agent(
    model=get_model("8B"),
    system_prompt="You are a helpful assistant that remembers our conversation. Be concise."
)

# Start a conversation
print("\nStarting conversation...")
response1 = persistent_agent("My name is Alice and I work as a data scientist at TechCorp.")
print(f"Response 1: {response1}")

In [ ]:
# Continue the conversation - agent remembers context
response2 = persistent_agent("What's my job title and where do I work?")
print(f"Response 2: {response2}")

In [ ]:
# Save the conversation to a file
session_file = os.path.join(session_dir, "alice_conversation.json")

# Agent messages are a list of dicts that can be serialized
conversation_data = {
    "agent_id": persistent_agent.agent_id,
    "messages": persistent_agent.messages
}

with open(session_file, "w") as f:
    json.dump(conversation_data, f, indent=2)

print(f"Conversation saved to: {session_file}")
print(f"Message count: {len(persistent_agent.messages)}")

In [ ]:
# Create a NEW agent and restore the conversation
restored_agent = Agent(
    model=get_model("8B"),
    system_prompt="You are a helpful assistant that remembers our conversation. Be concise."
)

# Load the saved conversation
with open(session_file, "r") as f:
    loaded_data = json.load(f)

# Restore messages to the new agent
restored_agent.messages = loaded_data["messages"]
print(f"Conversation restored with {len(restored_agent.messages)} messages")

# Test that context is preserved - the agent should remember Alice's name
response3 = restored_agent("What was my name again?")
print(f"Response from restored agent: {response3}")

---

## 7. Multi-Agent Orchestration

Chain multiple agents together for complex workflows. Each agent specializes in a specific task.

---

In [ ]:
# Create a content pipeline with specialized agents

# Fast classifier using smallest model
classifier = Agent(
    model=get_model("3B"),
    system_prompt="""Classify the input text into exactly ONE category:
    - TECHNICAL: Technical or scientific content
    - BUSINESS: Business, finance, or corporate content  
    - CREATIVE: Creative writing, stories, or artistic content
    - GENERAL: Everything else
    
    Respond with ONLY the category name, nothing else."""
)

# Specialized processors
technical_expert = Agent(
    model=get_model("14B"),
    system_prompt="""You are a technical expert. Analyze the content and provide:
    1. Key technical concepts identified
    2. Technologies or methods mentioned
    3. A clear technical summary
    Be thorough but concise."""
)

business_expert = Agent(
    model=get_model("8B"),
    system_prompt="""You are a business analyst. Analyze the content and provide:
    1. Key metrics and numbers
    2. Business insights and implications
    3. Actionable recommendations
    Format as an executive summary."""
)

creative_expert = Agent(
    model=get_model("14B"),
    system_prompt="""You are a creative writing expert. Analyze the content and provide:
    1. Writing style and tone analysis
    2. Literary devices or techniques used
    3. Suggestions for improvement
    Be constructive and insightful."""
)

general_expert = Agent(
    model=get_model("8B"),
    system_prompt="""Summarize the main points of the content clearly and concisely.
    Provide key takeaways that would be useful to the reader."""
)

print("Content Pipeline Agents created:")
print("  - Classifier (Ministral 3B) - fast routing")
print("  - Technical Expert (Ministral 14B)")
print("  - Business Expert (Ministral 8B)")
print("  - Creative Expert (Ministral 14B)")
print("  - General Expert (Ministral 8B)")

In [ ]:
def process_content(content: str) -> dict:
    """Process content through the multi-agent pipeline."""
    
    # Step 1: Classify the content
    print("Step 1: Classifying content...")
    category = str(classifier(content)).strip().upper()
    print(f"  Category: {category}")
    
    # Step 2: Route to appropriate expert
    print("Step 2: Routing to expert...")
    
    if "TECHNICAL" in category:
        expert = technical_expert
        expert_name = "Technical Expert"
    elif "BUSINESS" in category:
        expert = business_expert
        expert_name = "Business Expert"
    elif "CREATIVE" in category:
        expert = creative_expert
        expert_name = "Creative Expert"
    else:
        expert = general_expert
        expert_name = "General Expert"
    
    print(f"  Routed to: {expert_name}")
    
    # Step 3: Get expert analysis
    print("Step 3: Getting expert analysis...")
    analysis = expert(f"Analyze this content:\n\n{content}")
    
    return {
        "category": category,
        "expert": expert_name,
        "analysis": str(analysis)
    }

In [ ]:
# Test with technical content
technical_content = """
The new microservices architecture uses Kubernetes for container orchestration,
with Redis for caching and PostgreSQL for persistent storage. The API layer
implements GraphQL with rate limiting and JWT authentication. We've achieved
99.9% uptime since deployment.
"""

print("Processing TECHNICAL content:")
print("=" * 60)
result = process_content(technical_content)
print(f"\nAnalysis:\n{result['analysis']}")

In [ ]:
# Test with business content
business_content = """
Q3 revenue increased 15% YoY to $4.2M. Customer acquisition cost decreased
by 20% while retention improved to 94%. The board approved expansion into
the European market starting Q1 next year, with an initial investment of $2M.
"""

print("Processing BUSINESS content:")
print("=" * 60)
result = process_content(business_content)
print(f"\nAnalysis:\n{result['analysis']}")

---

## 8. Practical Use Case: Expert Panel

Create an expert panel where specialists from different fields analyze a topic and a moderator synthesizes their perspectives.

---

In [ ]:
# Create expert panel agents
economist = Agent(
    model=get_model("14B"),
    system_prompt="""You are an economist. Analyze topics from an economic perspective.
    Focus on: market dynamics, costs/benefits, incentives, and economic impacts.
    Provide 2-3 key insights. Be analytical and data-focused."""
)

technologist = Agent(
    model=get_model("14B"),
    system_prompt="""You are a technology expert. Analyze topics from a tech perspective.
    Focus on: technical feasibility, innovation potential, implementation challenges.
    Provide 2-3 key insights. Be practical and forward-thinking."""
)

ethicist = Agent(
    model=get_model("8B"),
    system_prompt="""You are an ethics specialist. Analyze topics from an ethical perspective.
    Focus on: societal impact, fairness, privacy, and moral considerations.
    Provide 2-3 key insights. Be thoughtful and balanced."""
)

moderator = Agent(
    model=get_model("large"),
    system_prompt="""You are a panel moderator. Your job is to:
    1. Synthesize different expert perspectives into a coherent summary
    2. Identify areas of agreement and disagreement
    3. Provide a balanced, actionable conclusion
    Be fair to all perspectives while providing clear guidance."""
)

print("Expert Panel created:")
print("  - Economist (Ministral 14B)")
print("  - Technologist (Ministral 14B)")
print("  - Ethicist (Ministral 8B)")
print("  - Moderator (Mistral Large 3)")

In [ ]:
def run_expert_panel(topic: str) -> str:
    """Run a full expert panel discussion on a topic."""
    
    print(f"Topic: {topic}")
    print("=" * 70)
    
    # Gather perspectives from each expert
    perspectives = {}
    
    print("\n--- Economist's Analysis ---")
    perspectives['economist'] = str(economist(f"Analyze this topic: {topic}"))
    print(perspectives['economist'])
    
    print("\n--- Technologist's Analysis ---")
    perspectives['technologist'] = str(technologist(f"Analyze this topic: {topic}"))
    print(perspectives['technologist'])
    
    print("\n--- Ethicist's Analysis ---")
    perspectives['ethicist'] = str(ethicist(f"Analyze this topic: {topic}"))
    print(perspectives['ethicist'])
    
    # Moderator synthesizes
    synthesis_prompt = f"""Topic: {topic}

Expert perspectives:

ECONOMIST: {perspectives['economist']}

TECHNOLOGIST: {perspectives['technologist']}

ETHICIST: {perspectives['ethicist']}

Please synthesize these perspectives into a balanced summary with clear conclusions."""
    
    print("\n--- Moderator's Synthesis ---")
    print("=" * 70)
    synthesis = str(moderator(synthesis_prompt))
    print(synthesis)
    
    return synthesis

In [ ]:
# Run the expert panel
topic = "The widespread adoption of AI in hiring and recruitment processes"
synthesis = run_expert_panel(topic)

---

## 9. Conclusion & Best Practices

---

### Model Selection Guidelines

| Task Type | Recommended Model | Reasoning |
|-----------|-------------------|------------|
| Quick Classification | **Ministral 3B** | Fastest response, lowest cost |
| General Conversations | **Ministral 8B** | Good balance of speed and quality |
| Complex Analysis | **Ministral 14B** | Higher accuracy for detailed work |
| Orchestration/Synthesis | **Mistral Large 3** | Best reasoning capabilities |

### Key Takeaways

1. **Start Simple**: Begin with a single agent, add complexity as needed
2. **Match Model to Task**: Use smaller models for simple tasks, larger for complex
3. **Stream for UX**: Use streaming for better user experience in interactive apps
4. **Monitor with Hooks**: Track latency and usage for optimization
5. **Persist Sessions**: Use session managers for stateful assistants
6. **Chain Agents**: Combine specialized agents for sophisticated workflows

### Architecture Patterns

| Pattern | Use Case | Example |
|---------|----------|----------|
| **Single Agent** | Simple Q&A, chat | Customer support bot |
| **Classifier + Experts** | Routing to specialists | Document processing |
| **Expert Panel** | Multi-perspective analysis | Decision support |
| **Pipeline** | Sequential processing | Content creation |

---

## Clean Up

This notebook uses Amazon Bedrock's serverless inference, so there are no persistent resources to clean up. You are charged based on token usage.

In [ ]:
# Clean up temporary session directory
import shutil

if 'session_dir' in dir() and os.path.exists(session_dir):
    shutil.rmtree(session_dir)
    print(f"Cleaned up session directory: {session_dir}")

print("\nNotebook complete!")